# EventIO Tutorial

The need to perform several actions concurrently arises frequently in microcontroller applications. Sensor outputs need to be read, motors and LEDs controlled, and buttons or touchpads checked for user input.

While it is possible in principle to perform all these actions sequentially in a loop, except in simple applications such solutions are difficult to get correct and even harder to maintain when new features are added.

This tutorial introduces `eventio`, a *partial* implementation of `curio`, a [library for concurrent processing](https://curio.readthedocs.io). 

## Blinking one LED

Let's start the tour with "blink", the proverbial "Hello World" application for microcontrollers. The code assumes that three LED's are connected to the board. Change the pin names to match your setup.

In [ ]:
%serialconnect "/dev/cu.SLAB_USBtoUART"

In [ ]:
from board import LED as LED_R
from machine import Pin
import time

def blink(color, pin, period):
    p = Pin(pin, mode=Pin.INOUT)
    for i in range(10):
        time.sleep(period/2)
        p.value(not p.value())
    
blink("red", LED_R, 0.5)

The code blinks the LED the specified number of times and then stops. Nothing unusual.

## Blinking multiple LEDs simulataneously

Now suppose we wanted to blink several LEDs at individual rates, say 0.7ms and 0.4ms periods. Simply calling calling `blink` twice with different parameters won't produce the desired effect:

In [ ]:
from board import A19 as LED_G
from board import A20 as LED_B

blink("red",   LED_R, 0.7)
blink("green", LED_G, 0.4)
blink("blue",  LED_B, 0.5)

This blinks the red light followed by the green light, one after the other. To blink them simultaneously but at different rates, modify `blink` as follows:

In [ ]:
import eventio

async def blink(color, pin, period):
    p = Pin(pin, mode=Pin.INOUT)
    for i in range(10):
        await eventio.sleep(period/2)
        p.value(not p.value())

The differences include importing the `eventio` library and peceeding the function definition with `async`. This marks it as a `coroutine`, a piece of code that can run concurrently with other operations. 

The call to `time.sleep` is replaced by `await eventio.sleep`. The keyword `await` tells the Python interpreter to perform other activity, turning on and off other LEDs in this case, simultaneously.

The following code blinkes the red, green, and blue LED concurrently:

In [ ]:
async def main():
    r = await eventio.spawn(blink, "red  ", LED_R, 0.7)
    g = await eventio.spawn(blink, "green", LED_G, 0.4)
    b = await eventio.spawn(blink, "blue ", LED_B, 0.5)
    
eventio.run(main)

The calls to `eventio.spawn` create three separate coroutines which run concurrently. The call to `eventio.run` starts the process.

In addition to the visual confirmation, we the example below adds printed output for further evidence of concurrent execution:

In [ ]:
%rebootdevice
%serialconnect "/dev/cu.SLAB_USBtoUART"

In [ ]:
from board import LED as LED_R
from board import A19 as LED_G
from board import A20 as LED_B
from machine import Pin
import eventio

chrono = eventio.Chronometer()

async def blink(color, pin, period):
    global chrono
    p = Pin(pin, mode=Pin.INOUT)
    last_time = 0
    for i in range(10):
        await eventio.sleep(period/2)
        p.value(not p.value())
        elapsed = chrono.elapsed_time
        delta = elapsed-last_time
        extra_delay = delta - period/2
        print("{} at {:6.0f} ms, {:4.0f} ms since last call, {:3.0f} ms extra".format( \
            color, 1000*elapsed, 1000*delta, 1000*extra_delay))
        last_time = chrono.elapsed_time
    print("{} is done".format(color))
    
async def main():
    r = await eventio.spawn(blink, "red  ", LED_R, 0.3)
    g = await eventio.spawn(blink, "green", LED_G, 0.7)
    b = await eventio.spawn(blink, "blue ", LED_B, 0.5)
    print("all LEDs blinking ...")
    await g.join()   # wait for coroutine g to terminate
    print("main is done")
    
eventio.run(main)

The output confirms that the coroutines indeed run simultataneously. It shows the times at which the LEDs are turned on or off. Analyzing them you will notice that each LED is called at multiples of half it's period, plus a "processing delay". Try to program all of this in a single loop ... And this is just for blinking LEDs; many real applications have more complex requirements!

Unlike calling `time.sleep`, which literally instructs the processor to do `nothing` except dissipate power and heat up the planet, a call to `eventio.sleep` checks for other work to be done, which in this example amounts to checking if one of the `blink` coroutines is ready to run. If no coroutine is ready to execute, `eventio.sleep` powers the processor down, which reduces the supply current by more than three orders-of-magnitude. 

**Note:** The "power down" feature requires an ARM CPU and is not available on the ESP32.

## Input

In the current implementation the LEDs blink for a fixed number of times. The code below stops when a button (connect a momentary button between `SWITCH` and ground) is pressed.

In [ ]:
%serialconnect "/dev/cu.SLAB_USBtoUART"

from board import LED as LED_R
from board import A19 as LED_G
from board import A20 as LED_B
from board import A5 as SWITCH
from machine import Pin, WDT
import eventio

WDT(False)

sw_event = eventio.PinEvent(SWITCH)

async def blink(color, pin, period):
    p = Pin(pin, mode=Pin.INOUT)
    try:
        while True:
            await eventio.sleep(period/2)
            p.value(not p.value())
    except eventio.CancelledError:
        print("{} cancelled".format(color))
        p.value(False)   # LED off
        raise

async def main():
    r = await eventio.spawn(blink, "red  ", LED_R, 0.3)
    g = await eventio.spawn(blink, "green", LED_G, 0.7)
    b = await eventio.spawn(blink, "blue ", LED_B, 0.5)
    print("All LEDs blinking ... press button to stop!")
    await sw_event.wait()
    print("Button pressed! Cancelling blinkers ...")
    await r.cancel()
    await g.cancel()
    await b.cancel()
    
eventio.run(main)

## Concurrency and Parallel Processing

Although it appears as if the coroutines run simultaneously, only one is active at any given time. This is similar to other computers (laptops, desktops, etc) which give the appearance of executing several programs concurrently but are in fact switching rapidly between different tasks (multi-core processors do run more than one program at the same time). If the processor alternates rapidly e.g. between playing a movie and a showing edits in a spreadsheet, it appears as if both were running simultaneously.

A big difference between the type of parallel processing used by laptop and desktop computers and `eventio` is the approach taken to switching between tasks. The former use a technique known as **preemptive multitasking**. In this case, the operating system (e.g. Linux or Windows) uses a timer to rapidly (e.g. every 50ms) switch between tasks: The currently running program is temporarily suspended, it's state saved, and a different task is permitted to execute. This process repeats, with each task getting a turn. If a task is blocked, e.g. waiting for input, it is skipped until it is ready again.

With preemptive multitasking, tasks are not aware when execution is interrupted and have no control over when this happens. By contrast, `eventio` uses a different form of task switching called **cooperative multitasking**. In this scenario, tasks have full control over when they relinquish the CPU, namely when calling `await`. In other words, a coroutine can be assured that no other task will interrupt it until it signals its "consent" with `await`. The duration of the suspension depends on the statement called and the tasks that are waiting. E.g. `await eventio.sleep(1)` suspends the task for at least one second, but could also suspend it for longer if other tasks use the CPU.

Preemptive and cooperative multitasking have both advantages and drawbacks, and which solution is better depends on the application. For small microcontrollers with limited memory and processing speed, cooperative multitasking is attractive since it uses less memory. More significantly, however, is the fact that task switching happens only at well defined instances marked with the keyword `await`.

One of the most challenging aspects of parallel processing is ensuring correct use of shared resources. For example, imagine reading the value from a sensor over `I2C`. The driver first sends the address of the sensor to get its attention. It subsequently asks the sensor return its value.

If this sequence is interrupted at an unfortunate moment by another task, the program may not function as intended. For example, if after addressing the sensor a different task gets control and addresses a sensor at another address, once control returns to the first task the request to read the sensor data goes to the wrong sensor producing an incorrect or no answer.

This type of error is very difficult to diagnose, as the error may occur only infrequently and is difficult to reproduce. With cooperative multitasking a task can be assured that it will not be interrupted unless calling `await`. In the above example, it would call `await` only after the sensor read operation has completed. While this does not address all challenges that arise with multitasking, cooperative multitasking substantially reduces the opportunities for errors over the alternative, preemptive multitasking.

The main drawback of cooperative multitasking is that a (rouge) task can prevent all others from running. E.g. if in the above example, the call to `eventio.sleep` is replaced by `time.sleep` (and the `await` keyword removed), the first coroutine will run exclusively until it finishes. In a desktop environment this obviously would not be practical - a simple programming mistake could freeze the entire computer. Microcontrollers, however, usually run a dedicated application that can be designed and tested for this not to happen.

## Caveats

In addition to the situation where a coroutine fails to call `await`, either due to a programming error or maliciously, thus preventing other tasks from running, there are a few other errors to avoid.

The first, and most vexing (the kind where everything _looks right_ but the program still refuses to operate as expected) is forgetting the `await` keyword, as in the example below:

In [ ]:
from board import LED as LED_R
from machine import Pin
import eventio

async def blink(color, pin, period):
    p = Pin(pin, mode=Pin.INOUT)
    # ERROR: no await!
    eventio.sleep(period/2)
    p.value(not p.value())
    
eventio.run(blink, "red", LED_R, 1)
print("Done!")

This program runs very quickly without error, but `eventio.sleep(period/2)` does not "sleep", hence resulting in the LED blinking very rapidly (faster than discernible).

The result of `eventio.sleep(1)` is a generator (or coroutine in some versions of Python):

In [ ]:
print(eventio.sleep(1))

It is the `await` keyword that instructs the interpreter to actually perform the function. Without the call to `eventio.sleep` simply produces the generator, which is discarded and control is passed to the next statement. Unfortunately in Python the statements with and without `await` are syntactically correct, so watch out!

The other common error is calling `time.sleep` instead of `eventio.sleep`. The former, with `await`, results in an error and is thus easy to spot. But without `await`, `time.sleep` is of course a valid call, except that it blocks the CPU, preventing other tasks from executing. 

If a concurrent program does not function as expected, these are some of the things to check!